In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import exifread

In [43]:
import os.path

In [3]:
f = file("/home/jbm/Desktop/20170517_jupiter_pics2/IMG_0655.CR2")
tags = exifread.process_file(f)
f.close()
len(tags)

161

In [6]:
[ k for k in tags.keys() if -1 != k.find("ate") ]

['EXIF DateTimeOriginal', 'Image DateTime', 'EXIF DateTimeDigitized']

In [8]:
x = tags['EXIF DateTimeOriginal']

In [11]:
x.values

u'2017:05:17 23:35:31'

In [12]:
[ k for k in tags.keys() if -1 != k.find("ocal") ]

['EXIF FocalPlaneResolutionUnit',
 'MakerNote FocalUnitsPerMM',
 'MakerNote FocalLength',
 'EXIF FocalLength',
 'MakerNote FocalType',
 'EXIF FocalPlaneYResolution',
 'EXIF FocalPlaneXResolution',
 'MakerNote LongFocalLengthOfLensInFocalUnits',
 'MakerNote ShortFocalLengthOfLensInFocalUnits']

In [15]:
tags['EXIF FocalLength'].values[0]

300

In [17]:
[ k for k in tags.keys() if -1 != k.find("xpo") ]

['MakerNote ExposureMode',
 'EXIF ExposureBiasValue',
 'EXIF ExposureTime',
 'EXIF ExposureMode',
 'MakerNote FlashExposureLock',
 'EXIF ExposureProgram',
 'MakerNote LongExposureNoiseReduction2']

In [20]:
tags['EXIF ExposureTime'].values[0]

1/40

In [21]:
[ k for k in tags.keys() if -1 != k.find("pertu") ]

['EXIF ApertureValue']

In [25]:
tags['EXIF ApertureValue'].values[0]

43/8

In [29]:
tags['EXIF ISOSpeedRatings'].values[0]

3200

In [28]:
[ k for k in tags.keys() if -1 != k.find("ISO") ]

['MakerNote ISO', 'EXIF ISOSpeedRatings']

In [44]:
def get_image_params(path):
    retval = {"path": path, "filename": os.path.basename(path)}
    
    f = file(path)
    tags = exifread.process_file(f)
    f.close()
    
    fields = {  # EXIF name => not-awful name
        'EXIF ISOSpeedRatings': 'ISO',
        'EXIF ApertureValue': 'aperture',
        'EXIF ExposureTime': 'exposure',
        'EXIF FocalLength': 'focal_length',
        'EXIF DateTimeOriginal': 'dt'
    }
    
    for k_e,k_human in fields.iteritems():
        v = tags[k_e].values
        if v.__class__ == list:
            v = v[0]
        retval[k_human] = v
        
    return retval

In [45]:
get_image_params("/home/jbm/Desktop/20170517_jupiter_pics2/IMG_0655.CR2")

{'ISO': 3200,
 'aperture': 43/8,
 'dt': u'2017:05:17 23:35:31',
 'exposure': 1/40,
 'filename': 'IMG_0655.CR2',
 'focal_length': 300,
 'path': '/home/jbm/Desktop/20170517_jupiter_pics2/IMG_0655.CR2'}

In [36]:
import glob

In [46]:
for p in glob.glob("/home/jbm/Desktop/20170517_jupiter_pics2/*.CR2"):
    x = get_image_params(p)
    print "%(filename)s: %(exposure)ss f-%(aperture)s at %(focal_length)smm, ISO %(ISO)s: %(dt)s" % x

IMG_0666.CR2: 1/40s f-43/8 at 300mm, ISO 3200: 2017:05:17 23:35:46
IMG_0740.CR2: 1/60s f-5 at 300mm, ISO 3200: 2017:05:17 23:39:56
IMG_0695.CR2: 1/50s f-5 at 300mm, ISO 3200: 2017:05:17 23:37:24
IMG_0792.CR2: 1/60s f-6 at 300mm, ISO 3200: 2017:05:17 23:43:24
IMG_0665.CR2: 1/40s f-43/8 at 300mm, ISO 3200: 2017:05:17 23:35:46
IMG_0752.CR2: 1/60s f-5 at 300mm, ISO 3200: 2017:05:17 23:40:15
IMG_0791.CR2: 1/60s f-6 at 300mm, ISO 3200: 2017:05:17 23:43:24
IMG_0743.CR2: 1/60s f-5 at 300mm, ISO 3200: 2017:05:17 23:40:02
IMG_0699.CR2: 1/50s f-5 at 300mm, ISO 3200: 2017:05:17 23:38:21
IMG_0777.CR2: 1/60s f-6 at 300mm, ISO 3200: 2017:05:17 23:41:14
IMG_0662.CR2: 1/40s f-43/8 at 300mm, ISO 3200: 2017:05:17 23:35:40
IMG_0776.CR2: 1/60s f-6 at 300mm, ISO 3200: 2017:05:17 23:41:13
IMG_0683.CR2: 1/50s f-5 at 300mm, ISO 3200: 2017:05:17 23:37:02
IMG_0722.CR2: 1/60s f-5 at 300mm, ISO 3200: 2017:05:17 23:39:23
IMG_0788.CR2: 1/60s f-6 at 300mm, ISO 3200: 2017:05:17 23:43:17
IMG_0770.CR2: 1/60s f-6 at 300m

In [47]:
from collections import defaultdict

In [58]:
def bucket_dir(dir_path):
    buckets = defaultdict(list) # bucket key => [ path ]
    
    bucket_keys = ["exposure", "aperture", "focal_length", "ISO"]
    
    for p in sorted(glob.glob("%s/*.CR2" % dir_path)):
        x = get_image_params(p)
        # print "%(filename)s: %(exposure)ss f-%(aperture)s at %(focal_length)smm, ISO %(ISO)s: %(dt)s" % x
    
        bk_l = [ (k, str(x[k])) for k in bucket_keys ]
        bk = tuple(bk_l)
        buckets[bk].append(p)
    
    return buckets
    

In [57]:
bucket_dir("/home/jbm/Desktop/20170517_jupiter_pics2/")

defaultdict(list,
            {(('exposure', '1/40'),
              ('aperture', '43/8'),
              ('focal_length', '300'),
              ('ISO',
               '3200')): ['/home/jbm/Desktop/20170517_jupiter_pics2/IMG_0666.CR2',
              '/home/jbm/Desktop/20170517_jupiter_pics2/IMG_0665.CR2',
              '/home/jbm/Desktop/20170517_jupiter_pics2/IMG_0662.CR2',
              '/home/jbm/Desktop/20170517_jupiter_pics2/IMG_0659.CR2',
              '/home/jbm/Desktop/20170517_jupiter_pics2/IMG_0673.CR2',
              '/home/jbm/Desktop/20170517_jupiter_pics2/IMG_0661.CR2',
              '/home/jbm/Desktop/20170517_jupiter_pics2/IMG_0656.CR2',
              '/home/jbm/Desktop/20170517_jupiter_pics2/IMG_0668.CR2',
              '/home/jbm/Desktop/20170517_jupiter_pics2/IMG_0657.CR2',
              '/home/jbm/Desktop/20170517_jupiter_pics2/IMG_0670.CR2',
              '/home/jbm/Desktop/20170517_jupiter_pics2/IMG_0667.CR2',
              '/home/jbm/Desktop/20170517_jupiter_pics2/